https://chatgpt.com/c/68ced95a-f138-832e-8e05-6ecd6d5f0c1d

In [ ]:
import os, numpy as np
import supervision as sv
from sports import MeasurementUnit, ViewTransformer
from sports.basketball import CourtConfiguration, League

import numpy as np

# 94x50 ft NBA court config (vertices known & ordered by the sports package)
CONFIG = CourtConfiguration(league=League.NBA, measurement_unit=MeasurementUnit.FEET)
print(CONFIG.vertices) 

# Example: use CONFIG.vertices as source and target (for demonstration)
source = np.array(CONFIG.vertices, dtype=np.float32)
target = np.array(CONFIG.vertices, dtype=np.float32)  # Replace with your actual target points

VT = ViewTransformer(source, target)
print(dir(VT))  # Lists all attributes and methods
print(vars(VT)) # Shows the object's __dict__ (attributes only, if available)


In [ ]:
# %% [markdown]
# https://chatgpt.com/c/68ced95a-f138-832e-8e05-6ecd6d5f0c1d

# %%
# !gdown --folder https://drive.google.com/drive/folders/1eDJYqQ77Fytz15tKGdJCMeYSgmoQ-2-H
# !gdown --folder https://drive.google.com/drive/folders/1RBjpI5Xleb58lujeusxH0W5zYMMA4ytO

# %%
from __future__ import annotations

import os
from pathlib import Path
from dataclasses import dataclass
from collections import deque
from typing import Union, Sequence, Optional, List

import cv2
import numpy as np
import supervision as sv
from tqdm import tqdm

# from inference import get_model
# from google.colab import userdata
from sports import MeasurementUnit, ViewTransformer
from sports.basketball import (
    CourtConfiguration,
    League,
    draw_court,
    draw_made_and_miss_on_court,
    ShotEventTracker,
    ShotEvent,  # imported but not used; kept for parity with original
    ShotType,   # imported but not used; kept for parity with original
)

# %%
# 94x50 ft NBA court config (vertices known & ordered by the sports package)
CONFIG = CourtConfiguration(league=League.NBA, measurement_unit=MeasurementUnit.FEET)
print(CONFIG.vertices)

# Example: use CONFIG.vertices as source and target (for demonstration)
source = np.array(CONFIG.vertices, dtype=np.float32)
target = np.array(CONFIG.vertices, dtype=np.float32)  # Replace with your actual target points

VT = ViewTransformer(source, target)
print(dir(VT))   # Lists all attributes and methods
print(vars(VT))  # Shows the object's __dict__ (attributes only, if available)

# %%
# config.py
from pathlib import Path as _Path  # (alias to avoid shadowing above)
import supervision as _sv
from sports import MeasurementUnit as _MeasurementUnit, ViewTransformer as _ViewTransformer
from sports.basketball import CourtConfiguration as _CourtConfiguration, League as _League
from inference import get_model as _get_model  # assumes available

src = _Path("api/src/cv/src")
court_image_dir = _Path("api/src/cv/data/images/court_detection/basketball-court-detection-2-1")
coordinates_dir = _Path("api/src/cv/data/coordinates")
video_dir = _Path("api/src/cv/data/videos")
boston_ex_video_path = video_dir / "boston-celtics-new-york-knicks-game-1"

SOURCE_VIDEO_PATH = _Path(
    "/workspace/api/src/cv/data/video/boston-celtics-new-york-knicks-game-1/"
    "boston-celtics-new-york-knicks-game-1-q1-03.16-03.11.mp4"
)

api_key = "htpcxp3XQh7SsgMfjJns"

PLAYER_DETECTION_MODEL_ID = "basketball-player-detection-3-ycjdo/4"
PLAYER_DETECTION_MODEL = _get_model(model_id=PLAYER_DETECTION_MODEL_ID, api_key=api_key)

COURT_DETECTION_MODEL_ID = "basketball-court-detection-2/14"
COURT_DETECTION_MODEL = _get_model(model_id=COURT_DETECTION_MODEL_ID, api_key=api_key)

# 94x50 ft NBA court config (vertices known & ordered by the sports package)
CONFIG = _CourtConfiguration(league=_League.NBA, measurement_unit=_MeasurementUnit.FEET)
print(CONFIG.vertices)

COLOR = sv.ColorPalette.from_hex([
    "#ffff00", "#ff9b00", "#ff66ff", "#3399ff", "#ff66b2", "#ff8080",
    "#b266ff", "#9999ff", "#66ffff", "#33ff99", "#66ff66", "#99ff00"
])
MAGENTA_COLOR = sv.Color.from_hex("#FF1493")
CYAN_COLOR = sv.Color.from_hex("#00BFFF")

CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.7
BALL_IN_BASKET_CLASS_ID = 1
JUMP_SHOT_CLASS_ID = 5
LAYUP_DUNK_CLASS_ID = 6

# %%
# from roboflow import Roboflow
# rf = Roboflow(api_key=api_key)
# project = rf.workspace("basketball-formations").project("basketball-court-detection-2-mlopt")
# version = project.version(1)
# dataset = version.download("yolov8")

# %% [markdown]
# # ball, number, player, referee and basket detection

# %%
print(f"current working directory: {os.getcwd()}")
# check on SOURCE_VIDEO_PATH
print(f"SOURCE_VIDEO_PATH: {SOURCE_VIDEO_PATH}")
print(f"COURT_DETECTION_MODEL: {COURT_DETECTION_MODEL}")

# test pull in the video
cap = cv2.VideoCapture(str(SOURCE_VIDEO_PATH))
ret, frame = cap.read()
print(f"frame (first read): {type(frame)}, shape={None if frame is None else frame.shape}")
cap.release()

box_annotator = sv.BoxAnnotator(color=COLOR, thickness=2)
label_annotator = sv.LabelAnnotator(color=COLOR, text_color=sv.Color.BLACK)

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH, start=65, iterative_seek=True)
frame = next(frame_generator)

result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.35)[0]
detections = sv.Detections.from_inference(result)

annotated_frame = frame.copy()
annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections)
sv.plot_image(annotated_frame)

# %% [markdown]
# detect jump-shots

# %%
box_annotator = sv.BoxAnnotator(color=COLOR, thickness=2)
label_annotator = sv.LabelAnnotator(color=COLOR, text_color=sv.Color.BLACK)

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH, start=65, iterative_seek=True)
frame = next(frame_generator)

result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.35)[0]
detections = sv.Detections.from_inference(result)
detections = detections[detections.class_id == 5]

annotated_frame = frame.copy()
annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections)
sv.plot_image(annotated_frame)

# %%
COLOR = sv.Color.from_hex("#007A33")
TEXT_COLOR = sv.Color.WHITE

triangle_annotator = sv.TriangleAnnotator(
    color=COLOR, base=25, height=21, color_lookup=sv.ColorLookup.INDEX
)
text_annotator = sv.RichLabelAnnotator(
    # font_path=f"{HOME}/fonts/Staatliches-Regular.ttf",
    font_size=60,
    color=COLOR,
    text_color=TEXT_COLOR,
    # text_offset=(0, -30),
    color_lookup=sv.ColorLookup.INDEX,
    text_position=sv.Position.TOP_CENTER,
)

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH, start=65, iterative_seek=True)
frame = next(frame_generator)

result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.35)[0]
detections = sv.Detections.from_inference(result)
detections = detections[detections.class_id == 5]

xy = detections.get_anchors_coordinates(anchor=sv.Position.BOTTOM_CENTER)

# small hack to convert points into detections
xyxy = sv.pad_boxes(np.hstack((xy, xy)), px=1, py=1)
detections = sv.Detections(xyxy=xyxy)
labels = ["here"] * len(detections)

annotated_frame = frame.copy()
annotated_frame = triangle_annotator.annotate(scene=annotated_frame, detections=detections)
annotated_frame = text_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)
sv.plot_image(annotated_frame)

# %% [markdown]
# detect basketball court keypoints

# %%
vertex_annotator = sv.VertexAnnotator(color=MAGENTA_COLOR, radius=8)

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH, start=65, iterative_seek=True)
frame = next(frame_generator)

result = COURT_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
key_points = sv.KeyPoints.from_inference(result)

annotated_frame = frame.copy()
annotated_frame = vertex_annotator.annotate(scene=annotated_frame, key_points=key_points)
sv.plot_image(annotated_frame)

# %%
import numpy as np
import supervision as sv

def filter_keypoints_by_confidence(
    key_points: sv.KeyPoints, min_conf: float, object_index: int = 0
) -> sv.KeyPoints:
    """
    Filter keypoints by confidence for a SINGLE object inside a Supervision KeyPoints.
    Returns a NEW KeyPoints with shape (1, k, 2) / (1, k); never mutates the input.

    - key_points.xy: expected (n, m, 2)
    - key_points.confidence: expected (n, m)
    - key_points.class_id: (n,) or None
    - key_points.data: dict-like (must NOT be None in your supervision version)

    Raises with explicit messages if shapes/fields are unexpected.
    """
    if key_points is None:
        raise ValueError("filter_keypoints_by_confidence: key_points is None.")
    if key_points.confidence is None:
        raise ValueError(
            "filter_keypoints_by_confidence: confidences are None; model did not return per-keypoint confidences."
        )

    xy = key_points.xy
    conf = key_points.confidence

    if xy.ndim != 3 or xy.shape[-1] != 2:
        raise ValueError(f"Expected xy shape (n, m, 2); got {xy.shape}.")
    if conf.ndim != 2:
        raise ValueError(f"Expected confidence shape (n, m); got {conf.shape}.")

    n, m, _ = xy.shape
    if conf.shape != (n, m):
        raise ValueError(f"xy/conf shape mismatch: xy {xy.shape}, conf {conf.shape}.")
    if not (0 <= object_index < n):
        raise IndexError(f"object_index {object_index} out of range for n={n}.")

    # Construct per-object mask along the m dimension
    obj_mask = conf[object_index] > float(min_conf)
    kept = int(obj_mask.sum())
    print(f"[DEBUG] objects: {n}, keypoints per obj: {m}, threshold: {min_conf}, kept(obj={object_index}): {kept}")

    # Slice to new arrays with shape (1, k, 2) and (1, k)
    new_xy = xy[object_index:object_index + 1, obj_mask, :]  # (1, k, 2)
    new_conf = conf[object_index:object_index + 1, obj_mask]  # (1, k)

    # Carry over class_id for the single object (shape (1,)) or None
    new_cls = None if key_points.class_id is None else key_points.class_id[object_index:object_index + 1]

    # Propagate original data if available; else use an empty dict.
    original_data = getattr(key_points, "data", None)
    new_data = original_data if isinstance(original_data, dict) else {}

    # If nothing passes, return an empty but well-formed KeyPoints
    if kept == 0:
        empty_xy = np.empty((1, 0, 2), dtype=new_xy.dtype)
        empty_conf = np.empty((1, 0), dtype=new_conf.dtype)
        return sv.KeyPoints(xy=empty_xy, confidence=empty_conf, class_id=new_cls, data=new_data)

    return sv.KeyPoints(xy=new_xy, confidence=new_conf, class_id=new_cls, data=new_data)


def sort_keypoints_by_class_id(key_points: sv.KeyPoints) -> sv.KeyPoints:
    """Return KeyPoints sorted by class_id if available; otherwise return as-is."""
    if key_points.class_id is None or len(key_points.class_id) == 0:
        return key_points
    idx = np.argsort(key_points.class_id)
    return key_points[idx]


def debug_dump_keypoints(key_points: sv.KeyPoints, limit: int = 10):
    """Print a small table of confidences and (x,y) for up to limit keypoints of each object."""
    xy = key_points.xy
    conf = key_points.confidence
    n, m, _ = xy.shape
    print(f"[DEBUG] dump: n={n}, m={m}, limit={limit}")
    for i in range(n):
        count = min(limit, m)
        print(f" [obj {i}] first {count} keypoints:")
        for j in range(count):
            cj = None if conf is None else float(conf[i, j])
            x, y = float(xy[i, j, 0]), float(xy[i, j, 1])
            print(f"  - idx={j:02d} | conf={cj:.4f} | (x,y)=({x:.1f},{y:.1f})")


if __name__ == "__main__":
    MAGENTA_COLOR = sv.Color(r=255, g=0, b=255)
    vertex_annotator = sv.VertexAnnotator(color=MAGENTA_COLOR, radius=8)

    frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH, start=65, iterative_seek=True)
    frame = next(frame_generator)

    result = COURT_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    print(f"result: {result}")

    key_points = sv.KeyPoints.from_inference(result)
    print("[DEBUG] xy shape:", key_points.xy.shape)
    print("[DEBUG] conf shape:", key_points.confidence.shape)
    print("[DEBUG] class_id shape:", None if key_points.class_id is None else key_points.class_id.shape)

    # NEW: filter within the single detection (object_index=0)
    key_points = filter_keypoints_by_confidence(key_points, min_conf=0.5, object_index=0)

    # Optional peek
    debug_dump_keypoints(key_points, limit=8)

    annotated_frame = frame.copy()
    annotated_frame = vertex_annotator.annotate(scene=annotated_frame, key_points=key_points)
    sv.plot_image(annotated_frame)

# %% [markdown]
# mark jump-shot location

# %%
CONFIG = CourtConfiguration(league=League.NBA, measurement_unit=MeasurementUnit.FEET)

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH, start=65, iterative_seek=True)
frame = next(frame_generator)

# detect court keypoints
court_result = COURT_DETECTION_MODEL.infer(frame, confidence=0.35)[0]
key_points = sv.KeyPoints.from_inference(court_result)

keypoint_mask = key_points.confidence[0] > 0.5
have_enough_points = np.count_nonzero(keypoint_mask) >= 4

if have_enough_points:
    court_vertices_masked = np.array(CONFIG.vertices)[keypoint_mask]          # (k, 2)
    detected_on_image = key_points.xy[0, keypoint_mask]                       # (k, 2)

    image_to_court = ViewTransformer(
        source=detected_on_image,
        target=court_vertices_masked,
    )

    # detect jump-shot
    result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.35)[0]
    detections = sv.Detections.from_inference(result)
    image_xy = detections[detections.class_id == 5].get_anchors_coordinates(anchor=sv.Position.BOTTOM_CENTER)

    if len(image_xy) > 0:
        court_xy = image_to_court.transform_points(points=image_xy)
        court = draw_made_and_miss_on_court(
            config=CONFIG,
            made_xy=court_xy,
            made_size=25,
            made_color=sv.Color.from_hex("#007A33"),
            made_thickness=6,
            line_thickness=4,
        )
        sv.plot_image(court)

# %% [markdown]
# detect shot events

# %%
box_annotator = sv.BoxAnnotator(color=COLOR, thickness=2)
label_annotator = sv.LabelAnnotator(color=COLOR, text_color=sv.Color.BLACK)

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

shot_event_tracker = ShotEventTracker(
    reset_time_frames=int(video_info.fps * 1.7),
    minimum_frames_between_starts=int(video_info.fps * 0.5),
    cooldown_frames_after_made=int(video_info.fps * 0.5),
)

for frame_index, frame in enumerate(frame_generator):
    result = PLAYER_DETECTION_MODEL.infer(
        frame, confidence=CONFIDENCE_THRESHOLD, iou_threshold=IOU_THRESHOLD
    )[0]
    detections = sv.Detections.from_inference(result)

    has_jump_shot = len(detections[detections.class_id == JUMP_SHOT_CLASS_ID]) > 0
    has_layup_dunk = len(detections[detections.class_id == LAYUP_DUNK_CLASS_ID]) > 0
    has_ball_in_basket = len(detections[detections.class_id == BALL_IN_BASKET_CLASS_ID]) > 0

    events = shot_event_tracker.update(
        frame_index=frame_index,
        has_jump_shot=has_jump_shot,
        has_layup_dunk=has_layup_dunk,
        has_ball_in_basket=has_ball_in_basket,
    )
    if events:
        print(events)

    annotated_frame = frame.copy()
    annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections)
    sv.plot_image(annotated_frame)

# %% [markdown]
# end-to-end multi-video processing

# %%
# Ensure HOME is defined (used by font paths / source dir below)
HOME = str(Path.home())

CONFIG = CourtConfiguration(league=League.NBA, measurement_unit=MeasurementUnit.FEET)
COLOR = sv.ColorPalette.from_hex(["#007A33", "#006BB6"])
TEXT_COLOR = sv.Color.WHITE

triangle_annotator = sv.TriangleAnnotator(
    color=COLOR, base=25, height=21, color_lookup=sv.ColorLookup.CLASS
)
text_annotator = sv.RichLabelAnnotator(
    # font_path=f"{HOME}/fonts/Staatliches-Regular.ttf",
    font_size=60,
    color=COLOR,
    text_color=TEXT_COLOR,
    # text_offset=(0, -30),
    color_lookup=sv.ColorLookup.CLASS,
    text_position=sv.Position.TOP_CENTER,
)

triangle_annotator_missed = sv.TriangleAnnotator(
    color=sv.Color.from_hex("#850101"), base=25, height=21, color_lookup=sv.ColorLookup.CLASS
)
text_annotator_missed = sv.RichLabelAnnotator(
    font_path=f"{HOME}/fonts/Staatliches-Regular.ttf",
    font_size=60,
    color=sv.Color.from_hex("#850101"),
    text_color=TEXT_COLOR,
    # text_offset=(0, -30),
    color_lookup=sv.ColorLookup.CLASS,
    text_position=sv.Position.TOP_CENTER,
)

class KeyPointsSmoother:
    def __init__(self, length: int):
        self.length = length
        self.buffer = deque(maxlen=length)

    def update(
        self,
        xy: np.ndarray,
        confidence: Optional[np.ndarray] = None,
        conf_threshold: float = 0.0,
    ) -> np.ndarray:
        assert xy.ndim == 3 and xy.shape[0] == 1
        xy_f = xy.astype(np.float32, copy=True)

        if confidence is not None:
            assert confidence.shape[:2] == xy.shape[:2]
            mask = (confidence >= conf_threshold)[..., None]
            xy_f = np.where(mask, xy_f, np.nan)

        self.buffer.append(xy_f)
        stacked = np.stack(self.buffer, axis=0)

        if np.isnan(stacked).any():
            mean_xy = np.nanmean(stacked, axis=0)
        else:
            mean_xy = stacked.mean(axis=0)

        return mean_xy


@dataclass
class Shot:
    x: float
    y: float
    distance: float
    result: bool
    team: int


def euclidean_distance(
    start_point: Union[Sequence[float], np.ndarray],
    end_point: Union[Sequence[float], np.ndarray],
) -> float:
    start_point_array = np.asarray(start_point, dtype=float)
    end_point_array = np.asarray(end_point, dtype=float)
    if start_point_array.shape != (2,) or end_point_array.shape != (2,):
        raise ValueError("Both points must have shape (2,).")
    return float(np.linalg.norm(end_point_array - start_point_array))


def extract_made(shots: List[Shot]):
    return [shot for shot in shots if shot.result]


def extract_xy(shots: List[Shot]) -> np.ndarray:
    return np.array([[shot.x, shot.y] for shot in shots], dtype=float)


def extract_class_id(shots: List[Shot]) -> np.ndarray:
    return np.array([shot.team for shot in shots], dtype=int)


def extract_label(shots: List[Shot]) -> np.ndarray:
    return np.array([f"{shot.distance:.2f} ft" for shot in shots], dtype=str)


SOURCE_VIDEO_DIR = Path(HOME) / "boston-celtics-new-york-knicks-game-1"
SOURCE_VIDEO_NAMES = [
    "boston-celtics-new-york-knicks-game-1-q1-03.16-03.11.mp4",
    "boston-celtics-new-york-knicks-game-1-q1-04.44-04.39.mp4",
    "boston-celtics-new-york-knicks-game-1-q1-06.00-05.54.mp4",
    "boston-celtics-new-york-knicks-game-1-q1-07.41-07.34.mp4",
]
SOURCE_VIDEO_PATHS = [SOURCE_VIDEO_DIR / video_name for video_name in SOURCE_VIDEO_NAMES]

KEYPOINT_CONFIDENCE_THRESHOLD = 0.5
DETECTION_CONFIDENCE = 0.3
CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.7
BALL_IN_BASKET_CLASS_ID = 1
JUMP_SHOT_CLASS_ID = 5
LAYUP_DUNK_CLASS_ID = 6
BALL_IN_BASKET_MIN_CONSECUTIVE_FRAMES = 2
JUMP_SHOT_MIN_CONSECUTIVE_FRAMES = 3
LAYUP_DUNK_MIN_CONSECUTIVE_FRAMES = 3

shots: List[Shot] = []

COURT_SCALE = 20
COURT_PADDING = 50
COURT_LINE_THICKNESS = 4

court_base = draw_court(
    config=CONFIG,
    scale=COURT_SCALE,
    padding=COURT_PADDING,
    line_thickness=COURT_LINE_THICKNESS,
)
court_h, court_w = court_base.shape[:2]

for video_path in tqdm(SOURCE_VIDEO_PATHS, desc="Videos", position=0):
    target_video_path = video_path.parent / f"{video_path.stem}-markers{video_path.suffix}"
    target_video_compressed_path = (
        target_video_path.parent / f"{target_video_path.stem}-compressed{target_video_path.suffix}"
    )
    target_court_video_path = video_path.parent / f"{video_path.stem}-court{video_path.suffix}"
    target_court_video_compressed_path = (
        target_court_video_path.parent
        / f"{target_court_video_path.stem}-compressed{target_court_video_path.suffix}"
    )

    video_info = sv.VideoInfo.from_video_path(str(video_path))
    total_frames = getattr(video_info, "total_frames", getattr(video_info, "frame_count", None))
    frame_generator = sv.get_video_frames_generator(str(video_path))

    court_video_info = sv.VideoInfo(
        width=court_w, height=court_h, fps=video_info.fps, total_frames=total_frames
    )

    shot_event_tracker = ShotEventTracker(
        reset_time_frames=int(video_info.fps * 1.7),
        minimum_frames_between_starts=int(video_info.fps * 0.5),
        cooldown_frames_after_made=int(video_info.fps * 0.5),
    )
    smoother = KeyPointsSmoother(length=3)
    shot_in_progress_xy: Optional[np.ndarray] = None

    with sv.VideoSink(str(target_video_path), video_info) as sink, \
         sv.VideoSink(str(target_court_video_path), court_video_info) as court_sink:

        for frame_index, frame in tqdm(
            enumerate(frame_generator),
            total=int(total_frames) if total_frames else None,
            desc=f"Frames: {video_path.name}",
            position=1,
            leave=False,
        ):
            # ================= Player detections and state ================
            player_result = PLAYER_DETECTION_MODEL.infer(
                frame, confidence=CONFIDENCE_THRESHOLD, iou_threshold=IOU_THRESHOLD
            )[0]
            player_detections = sv.Detections.from_inference(player_result)

            has_jump_shot = len(player_detections[player_detections.class_id == JUMP_SHOT_CLASS_ID]) > 0
            has_layup_dunk = len(player_detections[player_detections.class_id == LAYUP_DUNK_CLASS_ID]) > 0
            has_ball_in_basket = len(player_detections[player_detections.class_id == BALL_IN_BASKET_CLASS_ID]) > 0

            events = shot_event_tracker.update(
                frame_index=frame_index,
                has_jump_shot=has_jump_shot,
                has_layup_dunk=has_layup_dunk,
                has_ball_in_basket=has_ball_in_basket,
            )

            # ================= Court keypoints and transforms =============
            court_result = COURT_DETECTION_MODEL.infer(frame, confidence=DETECTION_CONFIDENCE)[0]
            key_points = sv.KeyPoints.from_inference(court_result)
            key_points.xy = smoother.update(
                xy=key_points.xy,
                confidence=key_points.confidence,
                conf_threshold=0.5,
            )

            key_mask = key_points.confidence[0] > KEYPOINT_CONFIDENCE_THRESHOLD
            have_enough_points = np.count_nonzero(key_mask) >= 4

            if have_enough_points:
                court_vertices_masked = np.array(CONFIG.vertices)[key_mask]
                detected_on_image = key_points.xy[0, key_mask]

                image_to_court = ViewTransformer(
                    source=detected_on_image,
                    target=court_vertices_masked,
                )
                court_to_image = ViewTransformer(
                    source=court_vertices_masked,
                    target=detected_on_image,
                )

            # ================= Events and global shot list =================
            if events:
                start_events = [e for e in events if e["event"] == "START"]
                made_events = [e for e in events if e["event"] == "MADE"]
                missed_events = [e for e in events if e["event"] == "MISSED"]

                if start_events and have_enough_points:
                    anchors_image = player_detections[
                        (player_detections.class_id == JUMP_SHOT_CLASS_ID)
                        | (player_detections.class_id == LAYUP_DUNK_CLASS_ID)
                    ].get_anchors_coordinates(anchor=sv.Position.BOTTOM_CENTER)

                    if len(anchors_image) > 0:
                        anchors_court = image_to_court.transform_points(points=anchors_image)
                        shot_in_progress_xy = anchors_court[0]

                if made_events and shot_in_progress_xy is not None:
                    shots.append(
                        Shot(
                            x=shot_in_progress_xy[0],
                            y=shot_in_progress_xy[1],
                            distance=euclidean_distance(
                                start_point=shot_in_progress_xy,
                                end_point=CONFIG.vertices[CONFIG.left_basket_index],
                            ),
                            result=True,
                            team=0,
                        )
                    )
                    shot_in_progress_xy = None

                if missed_events and shot_in_progress_xy is not None:
                    shots.append(
                        Shot(
                            x=shot_in_progress_xy[0],
                            y=shot_in_progress_xy[1],
                            distance=euclidean_distance(
                                start_point=shot_in_progress_xy,
                                end_point=CONFIG.vertices[CONFIG.left_basket_index],
                            ),
                            result=False,
                            team=0,
                        )
                    )
                    shot_in_progress_xy = None

            # ================= Render broadcast overlay ===================
            annotated = frame.copy()

            if have_enough_points and len(shots) > 0:
                made_shots = [s for s in shots if s.result is True]
                missed_shots = [s for s in shots if s.result is False]

                if len(made_shots) > 0:
                    made_xy_court = extract_xy(shots=made_shots)
                    made_xy_image = court_to_image.transform_points(points=made_xy_court)
                    boxes_xyxy_made = sv.pad_boxes(np.hstack((made_xy_image, made_xy_image)), px=1, py=1)
                    classes_made = extract_class_id(shots=made_shots)
                    detections_made = sv.Detections(xyxy=boxes_xyxy_made, class_id=classes_made)
                    labels_made = [f"{int(shot.distance)} feet" for shot in made_shots]

                    annotated = triangle_annotator.annotate(scene=annotated, detections=detections_made)
                    annotated = text_annotator.annotate(
                        scene=annotated, detections=detections_made, labels=labels_made
                    )

                if len(missed_shots) > 0:
                    missed_xy_court = extract_xy(shots=missed_shots)
                    missed_xy_image = court_to_image.transform_points(points=missed_xy_court)
                    boxes_xyxy_missed = sv.pad_boxes(np.hstack((missed_xy_image, missed_xy_image)), px=1, py=1)
                    classes_missed = extract_class_id(shots=missed_shots)
                    detections_missed = sv.Detections(xyxy=boxes_xyxy_missed, class_id=classes_missed)
                    labels_missed = ["missed"] * len(missed_shots)

                    annotated = triangle_annotator_missed.annotate(scene=annotated, detections=detections_missed)
                    annotated = text_annotator_missed.annotate(
                        scene=annotated, detections=detections_missed, labels=labels_missed
                    )

            # Write anno
